<h1 style="font-family : Arial; background-color : rgb(255,102,0); color : white; border : 2px solid; text-align:center; padding-top : 20px; padding-bottom:20px">REITERATION des CONVERSATIONS par MESSAGES</h1>

In [1]:
import os
chemin_dossier = 'C:\\Users\\sst\\Downloads\\REITERATION\\source_conversation'
fichiers = os.listdir(chemin_dossier)
nom_du_fichier = "réitération_avril"
intervalle = '60T'
n = 3

In [2]:
fichiers

['coversation_202404.xlsx']

In [3]:
import pandas as pd
fichier_excel = [fichier for fichier in fichiers if fichier.endswith('.xlsx')]
dfs = []
for fichier_excel in fichier_excel :
    chemin_fichier = os.path.join(chemin_dossier,fichier_excel)
    df = pd.read_excel(chemin_fichier)
    dfs.append(df)
data_final = pd.concat(dfs,ignore_index=True)
data_final

,source_name,created_at,first_content_author_id
0,Message Social Media Alpha,2024-04-01 00:46:51,100001383abzsdfhqsd
1,Message Social Media Alpha,2024-04-01 01:53:34,100001423abzsdfhqsd
2,Message Social Media Alpha,2024-04-01 02:12:56,100001054abzsdfhqsd
3,Message Social Media Alpha,2024-04-01 02:51:11,100001424abzsdfhqsd
4,Message Social Media Alpha,2024-04-01 04:43:50,100001339abzsdfhqsd
...,...,...,...
5560,Message Social Media Alpha,2024-05-02 22:54:02,100003506abzsdfhqsd
5561,Message Social Media Alpha,2024-05-02 23:00:08,100000131abzsdfhqsd
5562,Message Social Media Alpha,2024-05-02 23:24:03,100003530abzsdfhqsd
5563,Message Social Media Alpha,2024-05-02 23:31:06,100000232abzsdfhqsd


In [4]:
def converti_date(data_final) :
    data_final['date'] = pd.to_datetime(data_final['created_at'], format='%Y-%m-%d %H:%M:%S').dt.date
    data_final['year_month'] = data_final['created_at'].dt.to_period('M')
    return data_final
data_final = converti_date(data_final)

In [5]:
data_final.head()

,source_name,created_at,first_content_author_id,date,year_month
0,Message Social Media Alpha,2024-04-01 00:46:51,100001383abzsdfhqsd,2024-04-01,2024-04
1,Message Social Media Alpha,2024-04-01 01:53:34,100001423abzsdfhqsd,2024-04-01,2024-04
2,Message Social Media Alpha,2024-04-01 02:12:56,100001054abzsdfhqsd,2024-04-01,2024-04
3,Message Social Media Alpha,2024-04-01 02:51:11,100001424abzsdfhqsd,2024-04-01,2024-04
4,Message Social Media Alpha,2024-04-01 04:43:50,100001339abzsdfhqsd,2024-04-01,2024-04


In [6]:
data_final_tranche = data_final.copy()
liste_date = list(data_final_tranche['date'].unique())
data_jour = []
for date in liste_date :
    data_jour.append(data_final_tranche[data_final_tranche['date']==date])
# data_jour

In [7]:
tranche = data_final.copy()
def transformation(tranche, intervalle='H'):
    tranche['created_at'] = pd.to_datetime(tranche['created_at'])
    tranche['year_month'] = tranche['created_at'].dt.to_period('M')
    
    hourly_counts = tranche.groupby([tranche['year_month'],
                                     pd.Grouper(key='created_at', freq='D'),
                                     pd.Grouper(key='created_at', freq=intervalle)])['first_content_author_id'].count()
    
    hourly_unique = tranche.groupby([tranche['year_month'],
                                     pd.Grouper(key='created_at', freq='D'),
                                     pd.Grouper(key='created_at', freq=intervalle)])['first_content_author_id'].nunique()
    
    source_name = tranche.groupby([tranche['year_month'],
                                 pd.Grouper(key='created_at', freq='D'),
                                 pd.Grouper(key='created_at', freq=intervalle)])['source_name'].nunique()

    def transform(grouped_data, name):
        grouped_data.index.names = ['Year_Month', 'Date', 'Hour']
        grouped_data = pd.DataFrame(grouped_data)
        grouped_data.columns = [name]
        grouped_data = grouped_data.reset_index()
        grouped_data['Hour'] = grouped_data['Hour'].dt.strftime('%H:%M:%S')
        grouped_data = grouped_data.set_index(['Year_Month', 'Date', 'Hour'])
        return grouped_data

    hourly_counts = transform(hourly_counts, 'Conversations')
    hourly_unique = transform(hourly_unique, 'Conversations_Unique')
    source_name = transform(source_name, 'Source_Name')

    data_tranche = pd.concat([hourly_counts, hourly_unique], axis=1)
    data_tranche['Reiteration'] = round(100 * ((data_tranche['Conversations'] - data_tranche['Conversations_Unique']) / data_tranche['Conversations']), 2)
    data_tranche = pd.concat([data_tranche, source_name], axis=1)
    
    return data_tranche

tranche = data_final.copy()
data_tranche = transformation(tranche)
data_tranche

Conversations  Conversations_Unique  \
Year_Month Date       Hour                                            
2024-04    2024-04-01 00:00:00              1                     1   
                      01:00:00              1                     1   
                      02:00:00              2                     2   
                      04:00:00              1                     1   
                      06:00:00             10                    10   
...                                       ...                   ...   
2024-05    2024-05-02 19:00:00             12                    12   
                      20:00:00              7                     7   
                      21:00:00              6                     6   
                      22:00:00              3                     3   
                      23:00:00              4                     4   

                                Reiteration  Source_Name  
Year_Month Date       Hour                                
2024-04    2024-04-01 00:00:00          0.0            1  
                      01:00:00          0.0            1  
                      02:00:00          0.0            1  
                      04:00:00          0.0            1  
                      06:00:00          0.0            1  
...                                     ...          ...  
2024-05    2024-05-02 19:00:00          0.0            1  
                      20:00:00          0.0            1  
                      21:00:00          0.0            1  
                      22:00:00          0.0            1  
                      23:00:00          0.0            1  

[669 rows x 4 columns]

In [8]:
data_final = converti_date(data_final)
list_source = list(data_final['source_name'].unique())
list_dataframes = []
for i in range(len(list_source)) :
    df_prog = data_final[data_final['source_name']==list_source[i]].copy()
    list_dataframes.append(df_prog)
list_dataframes

[                     source_name          created_at first_content_author_id  \
 0     Message Social Media Alpha 2024-04-01 00:46:51     100001383abzsdfhqsd   
 1     Message Social Media Alpha 2024-04-01 01:53:34     100001423abzsdfhqsd   
 2     Message Social Media Alpha 2024-04-01 02:12:56     100001054abzsdfhqsd   
 3     Message Social Media Alpha 2024-04-01 02:51:11     100001424abzsdfhqsd   
 4     Message Social Media Alpha 2024-04-01 04:43:50     100001339abzsdfhqsd   
 ...                          ...                 ...                     ...   
 5560  Message Social Media Alpha 2024-05-02 22:54:02     100003506abzsdfhqsd   
 5561  Message Social Media Alpha 2024-05-02 23:00:08     100000131abzsdfhqsd   
 5562  Message Social Media Alpha 2024-05-02 23:24:03     100003530abzsdfhqsd   
 5563  Message Social Media Alpha 2024-05-02 23:31:06     100000232abzsdfhqsd   
 5564  Message Social Media Alpha 2024-05-02 23:34:33     100003531abzsdfhqsd   
 
             date year_mon

In [9]:
def triage(data_final) :
    pd.options.mode.copy_on_write = True
    reiteration = data_final['date'].unique()
    reiteration = pd.Series(reiteration)
    callers = []
    for i in range(len(reiteration)) :
        jours_glissant = reiteration[i:i+n]
        if len(jours_glissant) < n :
            break
        jours_glissant = list(jours_glissant)
        callers.append(jours_glissant)
    n_callers = []
    for i in range(len(callers)) :
        n_callers.append(data_final[data_final['date'].isin(callers[i])])
    n_unique = [caller[-1] for caller in callers]
    for caller, date in zip(n_callers, n_unique):
        caller['date'] = date
    n_callers = pd.concat(n_callers)
    reiteration = n_callers.set_index('date')
    return reiteration

In [10]:
top_contact = triage(data_final)
top_contact = top_contact.groupby(level=0)['first_content_author_id'].value_counts()
top_contact = pd.DataFrame(top_contact)
top_contact = top_contact.rename(columns={'first_content_author_id':'count'}) 
top_contact = top_contact.reset_index()
top_contact = top_contact[top_contact['count']>=2]
top_contact = top_contact.sort_values(by=['count'],ascending=False)
top_unique = top_contact['first_content_author_id'].unique()
top_contact_unique = data_final[data_final['first_content_author_id'].isin(top_unique)]
top_contact_unique = top_contact_unique.sort_values(by=['first_content_author_id','created_at'],ascending=True).drop(columns=['date'])
top_contact_unique

,source_name,created_at,first_content_author_id,year_month
3417,Message Social Media Alpha,2024-04-16 16:03:16,100000002abzsdfhqsd,2024-04
3757,Message Social Media Alpha,2024-04-18 17:30:59,100000002abzsdfhqsd,2024-04
111,Message Social Media Alpha,2024-04-01 16:58:00,100000004abzsdfhqsd,2024-04
739,Message Social Media Alpha,2024-04-03 21:29:02,100000004abzsdfhqsd,2024-04
1350,Message Social Media Alpha,2024-04-06 06:48:39,100000004abzsdfhqsd,2024-04
...,...,...,...,...
5452,Message Social Media Alpha,2024-05-02 09:26:14,100003493abzsdfhqsd,2024-05
5542,Message Social Media Alpha,2024-05-02 19:28:27,100003493abzsdfhqsd,2024-05
5478,Message Social Media Alpha,2024-05-02 12:07:21,100003506abzsdfhqsd,2024-05
5530,Message Social Media Alpha,2024-05-02 18:32:01,100003506abzsdfhqsd,2024-05


In [12]:
reiteration = triage(data_final)
reiteration_par_source = []
for i in range(len(list_dataframes)) :
    reit_p = reiteration[reiteration['source_name']==list_source[i]].copy()
    reiteration_par_source.append(reit_p)
reiteration_par_source

[                           source_name          created_at  \
 date                                                         
 2024-04-03  Message Social Media Alpha 2024-04-01 00:46:51   
 2024-04-03  Message Social Media Alpha 2024-04-01 01:53:34   
 2024-04-03  Message Social Media Alpha 2024-04-01 02:12:56   
 2024-04-03  Message Social Media Alpha 2024-04-01 02:51:11   
 2024-04-03  Message Social Media Alpha 2024-04-01 04:43:50   
 ...                                ...                 ...   
 2024-05-02  Message Social Media Alpha 2024-05-02 22:54:02   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:00:08   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:24:03   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:31:06   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:34:33   
 
            first_content_author_id year_month  
 date                                           
 2024-04-03     100001383abzsdfhqsd    2024-04  
 2024-04-03     100001423abzsdfh

In [13]:
reiteration.head()

,source_name,created_at,first_content_author_id,year_month
date,,,,
2024-04-03,Message Social Media Alpha,2024-04-01 00:46:51,100001383abzsdfhqsd,2024-04
2024-04-03,Message Social Media Alpha,2024-04-01 01:53:34,100001423abzsdfhqsd,2024-04
2024-04-03,Message Social Media Alpha,2024-04-01 02:12:56,100001054abzsdfhqsd,2024-04
2024-04-03,Message Social Media Alpha,2024-04-01 02:51:11,100001424abzsdfhqsd,2024-04
2024-04-03,Message Social Media Alpha,2024-04-01 04:43:50,100001339abzsdfhqsd,2024-04


In [14]:
def resultat(reiteration) :
    temp = reiteration.drop(columns=['source_name','created_at','first_content_author_id','year_month'])
    temp = temp.groupby(level=0).value_counts()
    temp = pd.DataFrame(temp)
    temp = temp.rename(columns={0:'Conversations'})
    reit = reiteration.drop(columns=['source_name','created_at'])
    reit = reit.groupby(level=0).nunique()
    Reiteration = pd.concat([temp,reit],axis=1)
    Reiteration = Reiteration.rename(columns={'first_content_author_id':'Conversations_Unique'})
    Reiteration['Reiteration'] = round(100*((Reiteration['Conversations']-Reiteration['Conversations_Unique'])/Reiteration['Conversations']),2)
    return Reiteration

In [15]:
list_dataframes

[                     source_name          created_at first_content_author_id  \
 0     Message Social Media Alpha 2024-04-01 00:46:51     100001383abzsdfhqsd   
 1     Message Social Media Alpha 2024-04-01 01:53:34     100001423abzsdfhqsd   
 2     Message Social Media Alpha 2024-04-01 02:12:56     100001054abzsdfhqsd   
 3     Message Social Media Alpha 2024-04-01 02:51:11     100001424abzsdfhqsd   
 4     Message Social Media Alpha 2024-04-01 04:43:50     100001339abzsdfhqsd   
 ...                          ...                 ...                     ...   
 5560  Message Social Media Alpha 2024-05-02 22:54:02     100003506abzsdfhqsd   
 5561  Message Social Media Alpha 2024-05-02 23:00:08     100000131abzsdfhqsd   
 5562  Message Social Media Alpha 2024-05-02 23:24:03     100003530abzsdfhqsd   
 5563  Message Social Media Alpha 2024-05-02 23:31:06     100000232abzsdfhqsd   
 5564  Message Social Media Alpha 2024-05-02 23:34:33     100003531abzsdfhqsd   
 
             date year_mon

In [16]:
reiteration_par_source

[                           source_name          created_at  \
 date                                                         
 2024-04-03  Message Social Media Alpha 2024-04-01 00:46:51   
 2024-04-03  Message Social Media Alpha 2024-04-01 01:53:34   
 2024-04-03  Message Social Media Alpha 2024-04-01 02:12:56   
 2024-04-03  Message Social Media Alpha 2024-04-01 02:51:11   
 2024-04-03  Message Social Media Alpha 2024-04-01 04:43:50   
 ...                                ...                 ...   
 2024-05-02  Message Social Media Alpha 2024-05-02 22:54:02   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:00:08   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:24:03   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:31:06   
 2024-05-02  Message Social Media Alpha 2024-05-02 23:34:33   
 
            first_content_author_id year_month  
 date                                           
 2024-04-03     100001383abzsdfhqsd    2024-04  
 2024-04-03     100001423abzsdfh

In [17]:
globales = resultat(reiteration)
resultat_par_source = list_dataframes.copy()
for i in range(len(list_dataframes)) :
    resultat_par_source[i] = resultat(reiteration_par_source[i])
    resultat_par_source[i]['source_name'] = list_source[i]
resultat_par_source

[            Conversations  Conversations_Unique  year_month  Reiteration  \
 date                                                                       
 2024-04-03            752                   597           1        20.61   
 2024-04-04            906                   735           1        18.87   
 2024-04-05            863                   714           1        17.27   
 2024-04-06            803                   679           1        15.44   
 2024-04-07            668                   567           1        15.12   
 2024-04-08            640                   537           1        16.09   
 2024-04-09            655                   551           1        15.88   
 2024-04-10            676                   560           1        17.16   
 2024-04-11            652                   503           1        22.85   
 2024-04-12            613                   462           1        24.63   
 2024-04-13            556                   431           1        22.48   

In [19]:
data_finals = data_final.drop(columns=['date','year_month'])
with pd.ExcelWriter("C:\\Users\\sst\\Downloads\\REITERATION\\"+nom_du_fichier+".xlsx") as writer :
    data_finals.to_excel(writer, sheet_name='Data',index=False)
    data_tranche.to_excel(writer,sheet_name='SummaryH',index=True)
    globales.to_excel(writer,sheet_name='SummaryD',index=True)
    top_contact.to_excel(writer,sheet_name='top_id_conversations',index=False)
    top_contact_unique.to_excel(writer,sheet_name='top_id_converstions_Details',index=False)
    for i in range(len(resultat_par_source)) :
        resultat_par_source[i].to_excel(writer,sheet_name=list_source[i],index=True)